In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.metrics import mean_squared_error, r2_score

In [5]:
data = pd.read_csv("../CSV/big_files/ais_train_processed.csv", sep="|")
data.drop(columns=["portId"], inplace=True)




,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,drift,at_port,time_at_sea
132618,1970-01-01 00:00:01.705079676,307.5,16.1,6,307,0,1705275000,8.03598,76.90095,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,0.5,False,11209.0
1461726,1970-01-01 00:00:01.714719274,344.8,0.7,-5,120,0,1714631400,41.05015,2.11713,61e9f38eb937134a3c4bfd8b,61d37f9c29b60f6113c89e65,224.8,False,9650807.0
1451898,1970-01-01 00:00:01.714647202,52.2,2.0,-3,170,0,1714631400,41.05180,2.45416,61e9f38eb937134a3c4bfd8b,61d37f9c29b60f6113c89e65,-117.8,False,9578735.0
664265,1970-01-01 00:00:01.709049286,82.8,0.6,0,43,1,1707912000,9.42835,-79.95952,61e9f38eb937134a3c4bfd8b,61d3784c93c6feb83e5eb7a2,39.8,False,3980819.0
1275280,1970-01-01 00:00:01.713382558,288.7,15.9,9,289,0,1714705200,-34.61280,18.40357,61e9f38eb937134a3c4bfd8b,61d37f9c29b60f6113c89e65,-0.3,False,8314091.0


In [ ]:
# --- Time Series KNN ---
from sktime.regression.distance_based import KNeighborsTimeSeriesRegressor

KNNTS = KNeighborsTimeSeriesRegressor(n_neighbors=3, distance=eucl_dist)